In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('ANNOTED_CSV_PDFS.csv')

df.head(7)

,Keywords,Title,cluster,categorie
0,software architecture architecture description...,A Framework for Classifying and Comparing Arch...,0,architecture software
1,software architecture architecture description...,A Language and Environment for Architecture-Ba...,0,architecture software
2,software architecture maturity models isoiecieee,A Maturity Model for Software Architecture bas...,0,architecture software
3,service oriented architecture mobile phones dy...,A Pluggable Architectural Model and a Formally...,0,architecture software
4,software architecture description languages re...,A Rewriting Semantics for a Software Architect...,0,architecture software
5,architectural description languages security m...,A Secure Architectural Description Language fo...,0,architecture software
6,language engineering nlp architecture uima,A Survey of Text Mining Architectures and the ...,0,architecture software


In [2]:
def research (word, radio_value) :
    if radio_value == 'keywords':
        mask = df['Keywords'].str.contains(word)
    elif radio_value == 'title': 
        mask = df['categorie'].str.contains(word)
    filtered_df = df[mask]
    return filtered_df.values.tolist()
data_list = research('languages', 'keywords')
data_list[0]
print(type(data_list))

<class 'list'>


In [3]:
def content_recommender(Kws, nbr_page_value, cat): #ajouter nb_kws pour le code final
    """
    Kws = ''
    for i in range(nb_kws):
        Kws += input("Entrez un mot-clé : ") + ' '
    """
    new_row = { 'Keywords': Kws, 'Title': 'keywordsSearch', 'cluster': '', 'categorie': ''}
    dff = df.append(new_row, ignore_index=True)

    tfidf = TfidfVectorizer(stop_words='english')
    dff['Keywords'] = dff['Keywords'].fillna('')
    if cat == "title":
        tfidf_matrix = tfidf.fit_transform(dff['Keywords'] + ' ' + dff['categorie'])
        
    else:
        tfidf_matrix = tfidf.fit_transform(dff['Keywords'])
    

    cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
    cosine_sime= cosine_similarity(tfidf_matrix,tfidf_matrix)
    cosine_sim1=cosine_sim[:,0:3]
    cosine_sim2=cosine_sime[:,0:3]
    indices = pd.Series(dff.index, index=dff['Title']).drop_duplicates()

    idx=indices['keywordsSearch']
    sim_scores= list(enumerate(cosine_sim[idx]))
    sim_scores= sorted(sim_scores, key=lambda x : x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    df_indices=[i[0] for i in sim_scores]
    df_percentage=[str(round(i[1]*100, 2))+'%' for i in sim_scores]
    
    res = dff['Title'].iloc[df_indices].to_frame()
    res.insert(1, 'Matching Score', df_percentage)
    res['Categorie'] = df['categorie']
    
    print("Recommendations: ")
    print(res.head(5))
   
    return res.head(nbr_page_value).values.tolist()

In [4]:
# load the dataset
dataset = pd.read_csv('anoted_df.csv')

# split the dataset into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# create a vectorizer to convert the keywords to a bag-of-words representation
vectorizer = CountVectorizer()

# fit the vectorizer to the training data
vectorizer.fit(train_data['Keywords'])

# transform the keywords in the training and testing sets to the bag-of-words representation
X_train = vectorizer.transform(train_data['Keywords'])
X_test = vectorizer.transform(test_data['Keywords'])

# convert the document titles to numerical labels
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Title'])
y_train = label_encoder.transform(train_data['Title'])
y_test = label_encoder.transform(test_data['Title'])

# define the deep learning model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Epoch 1/100


D:\anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


2/2 [==============================] - 1s 262ms/step - loss: 4.1473 - accuracy: 0.0000e+00 - val_loss: 4.1348 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 12ms/step - loss: 4.1101 - accuracy: 0.0204 - val_loss: 4.1408 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 13ms/step - loss: 4.0761 - accuracy: 0.0204 - val_loss: 4.1469 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 13ms/step - loss: 4.0439 - accuracy: 0.0204 - val_loss: 4.1529 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 12ms/step - loss: 4.0135 - accuracy: 0.0408 - val_loss: 4.1588 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 14ms/step - loss: 3.9852 - accuracy: 0.0612 - val_loss: 4.1648 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 12ms/step - loss: 3.9553 - accuracy: 0.1020 - val_loss: 4.1709 - val_accuracy: 0.0000e+00

In [5]:
# load the dataset
dataset = pd.read_csv('anoted_df.csv')

def fusionner_deux_colonnes(colonne1, colonne2):
    return colonne1 + ' ' + colonne2

# appliquer la fonction à chaque ligne de la dataframe pour créer une nouvelle colonne
dataset['KeywordsAndCat'] = df.apply(lambda x: fusionner_deux_colonnes(x['Keywords'], x['categorie']), axis=1)


# split the dataset into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# create a vectorizer to convert the keywords to a bag-of-words representation
vectorizer = CountVectorizer()

# fit the vectorizer to the training data
vectorizer.fit(train_data['KeywordsAndCat'])

# transform the keywords in the training and testing sets to the bag-of-words representation
X_train = vectorizer.transform(train_data['KeywordsAndCat'])
X_test = vectorizer.transform(test_data['KeywordsAndCat'])

# convert the document titles to numerical labels
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Title'])
y_train = label_encoder.transform(train_data['Title'])
y_test = label_encoder.transform(test_data['Title'])

# define the deep learning model
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
history2 = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)


Epoch 1/100
2/2 [==============================] - 0s 20ms/step - loss: 0.1590 - accuracy: 1.0000 - val_loss: 9.1127 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 13ms/step - loss: 0.1479 - accuracy: 1.0000 - val_loss: 9.0997 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 11ms/step - loss: 0.1355 - accuracy: 1.0000 - val_loss: 9.0778 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 12ms/step - loss: 0.1223 - accuracy: 1.0000 - val_loss: 9.0524 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 13ms/step - loss: 0.1114 - accuracy: 1.0000 - val_loss: 9.0309 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 13ms/step - loss: 0.1023 - accuracy: 1.0000 - val_loss: 9.0136 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 12ms/step - loss: 0.0966 - accuracy: 1.0000 - val_loss: 8.9975 - val_accuracy: 0.0

In [6]:
def predict_titles(keywords, nbr_page_value, cat):
    # transform the keywords to the bag-of-words representation
    X = vectorizer.transform([keywords]).toarray()
    # predict the title of the document using the trained model
    if cat == "Title":
        y_pred = model2.predict(X)
    else :
        y_pred = model.predict(X)
    # calculate the matching scores for all titles
    scores = np.dot(model.get_weights()[0].T, X.T).ravel()
    # sort the titles by their matching scores and return the top 5
    top_indices = np.argsort(scores)[-5:][::-1]
    # convert the predicted labels to the corresponding titles
    predicted_titles = label_encoder.inverse_transform(top_indices)
    cat = dataset[dataset['Title'].isin(predicted_titles)]['categorie'].tolist()
    probabilities = [f'{float(score)*100:.2f} %' for score in scores[top_indices]]
    df = pd.DataFrame({'title': predicted_titles, 'matching score': probabilities, 'categories': cat})
    return df.head(nbr_page_value).values.tolist()


In [7]:
from flask import Flask, request, render_template, jsonify, redirect, url_for

app = Flask(__name__)


@app.route('/')
def index():
    return render_template('index.html')




@app.route('/pdfTool', methods=['POST'])
def pdfTool():
    
    search_word = request.form['query']
    print(search_word)
    filter_serch = request.form['search_type']
    print(filter_serch)
    reco_value = request.form['reco_type']
    print(reco_value)
    nbr_page_value = request.form['nbre_pages']
    print(nbr_page_value)
    
    if reco_value == "DL":

        employee = predict_titles(search_word, int(nbr_page_value), filter_serch)
    else:
        employee = content_recommender(search_word, int(nbr_page_value), filter_serch)
    
    return jsonify({'htmlresponse': render_template('reponses.html', employee=employee)})

In [8]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Mar/2023 15:46:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2023 15:46:35] "POST /pdfTool HTTP/1.1" 400 -
127.0.0.1 - - [20/Mar/2023 15:46:35] "GET /favicon.ico HTTP/1.1" 404 -


networking vanet protocols standards routing security broadcasting simulation
keywords
DL
3
networking vanet protocols standards routing security broadcasting simulation
keywords
DL
3
1/1 [==============================] - 0s 148ms/step


127.0.0.1 - - [20/Mar/2023 15:48:47] "POST /pdfTool HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2023 15:48:47] "POST /pdfTool HTTP/1.1" 200 -


networking vanet protocols  routing security broadcasting simulation
keywords
DL
3
1/1 [==============================] - 0s 34ms/step


...

networking vanet protocols routing
title
BC
4
Recommendations: 
                                                Title Matching Score  \
4   Analytical Model for Connectivity in Vehicular...         63.35%   
11           A_Survey_of_Vehicular_Ad_hoc_Networks_Ro         32.46%   
8   Asurvey and challenges in routing and data dis...         30.91%   
27  Vehicular Ad Hoc Networks (VANETs)  An Overvie...         29.97%   
9            A_comprehensive_survey_on_vehicular_Ad_H          29.5%   

            Categorie  
4   vehicular network  
11  vehicular network  
8   vehicular network  
27  vehicular network  
9   vehicular network  


127.0.0.1 - - [20/Mar/2023 15:55:17] "POST /pdfTool HTTP/1.1" 200 -


networking vanet protocols routing
title
BC
4
Recommendations: 
                                                Title Matching Score  \
4   Analytical Model for Connectivity in Vehicular...         63.35%   
11           A_Survey_of_Vehicular_Ad_hoc_Networks_Ro         32.46%   
8   Asurvey and challenges in routing and data dis...         30.91%   
27  Vehicular Ad Hoc Networks (VANETs)  An Overvie...         29.97%   
9            A_comprehensive_survey_on_vehicular_Ad_H          29.5%   

            Categorie  
4   vehicular network  
11  vehicular network  
8   vehicular network  
27  vehicular network  
9   vehicular network  


127.0.0.1 - - [20/Mar/2023 15:55:18] "POST /pdfTool HTTP/1.1" 200 -


networking vanet protocols routing
title
BC
4
Recommendations: 
                                                Title Matching Score  \
4   Analytical Model for Connectivity in Vehicular...         63.35%   
11           A_Survey_of_Vehicular_Ad_hoc_Networks_Ro         32.46%   
8   Asurvey and challenges in routing and data dis...         30.91%   
27  Vehicular Ad Hoc Networks (VANETs)  An Overvie...         29.97%   
9            A_comprehensive_survey_on_vehicular_Ad_H          29.5%   

            Categorie  
4   vehicular network  
11  vehicular network  
8   vehicular network  
27  vehicular network  
9   vehicular network  


from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/')
def search():
    return render_template('results.html')

@app.route('/results')
def results():
    query = request.args.get('query')
    # process the query and get the results
    results = process_query(query)
    return render_template('results.html', results=results)

def process_query(query):
    # write code here to get the results for the given query
    # ...
    return results





